In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sys

sys.path.append("/home/dchen/BA_CH_EN/")

# Basics
import tensorflow as tf
import xarray as xr
import pandas as pd

# Helpful
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import *  # DRN implementation
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores
from src.models.U_net.unet import Unet
from src.models.U_net.u_net_tests_params import *
from src.models.U_net.u_net_load_data import *

2023-07-04 16:48:54.104565: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-07-04 16:48:54.104613: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 0. Setup

In [3]:
var_num = 2
lead_time = 0

<IPython.core.display.Javascript object>

### 1. Load Data

In [15]:
(
    padded_train_data_mean,
    padded_train_data_std,
    padded_train_data_y,
) = u_net_load_train_data(2, 0)

<IPython.core.display.Javascript object>

In [16]:
padded_test_data_mean, padded_test_data_std, padded_test_data_y = u_net_load_train_data(
    2, 0
)

<IPython.core.display.Javascript object>

In [20]:
# Parameters for Unet class initialization
v = "tp"  # Or any other value based on your preference
train_patches = False  # Or True if you want to train with patches

# Initialize the U-Net model
unet_model = Unet(v=v, train_patches=train_patches, filters=16)

# Build the model with your training data shape
model = unet_model.build_model(padded_train_data_mean.shape, 2, learning_rate=0.01)

Downsampling block 1 shape:  (None, 64, 72, 64)
Downsampling block 2 shape:  (None, 32, 36, 128)
Downsampling block 3 shape:  (None, 16, 18, 256)
Downsampling block 4 shape:  (None, 8, 9, 512)
Bottleneck shape:  (None, 8, 9, 1024)
Upsampling block 5 shape:  (None, 8, 9, 1024)
Upsampling block 4 shape:  (None, 16, 18, 512)
Upsampling block 3 shape:  (None, 32, 36, 256)
Upsampling block 2 shape:  (None, 64, 72, 128)
Upsampling block 1 shape:  (None, 128, 144, 64)


<IPython.core.display.Javascript object>

In [21]:
hist = model.fit(
    [padded_train_data_mean, padded_train_data_std],
    padded_train_data_y,
    epochs=150,
    shuffle=True,
    batch_size=128,
    validation_split=0.25,
)

Epoch 1/150
Var equals 0
Var equals 0


2023-07-04 16:54:50.230302: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 603979776 exceeds 10% of free system memory.
2023-07-04 16:54:50.342259: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 603979776 exceeds 10% of free system memory.
2023-07-04 16:54:50.492390: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 603979776 exceeds 10% of free system memory.
2023-07-04 16:54:53.151911: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 603979776 exceeds 10% of free system memory.
2023-07-04 16:54:53.593639: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 603979776 exceeds 10% of free system memory.


9/9 [==============================] - 213s 23s/step - loss: 281.8197 - val_loss: 311836.6562
Epoch 2/150
9/9 [==============================] - 211s 23s/step - loss: 277.3815 - val_loss: 12189.4473
Epoch 3/150
9/9 [==============================] - 210s 23s/step - loss: 271.6341 - val_loss: 2056.6323
Epoch 4/150
9/9 [==============================] - 210s 23s/step - loss: 263.7944 - val_loss: 978.7692
Epoch 5/150
9/9 [==============================] - 211s 24s/step - loss: 253.4721 - val_loss: 585.6443
Epoch 6/150
2/9 [=====>........................] - ETA: 2:44 - loss: 245.2960

KeyboardInterrupt: 

<IPython.core.display.Javascript object>